In [6]:
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

In [7]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Set things up

In [4]:
import caveclient as cc
import importlib.metadata

print(f"CAVEclient version: v{cc.__version__} , v{importlib.metadata.version('CAVEclient')}")

datastack_name = "minnie65_phase3_v1"

client = cc.CAVEclient(datastack_name)
client.materialize.version = 1078

## Pick a test server:
* **localhost:5000 — Test SkeletonService on the local machine, say via the VS Code Debugger**
* **ltv5 — The SkeletonService on the test cluster**
* **minniev6 — Test SkeletonService "in the wild"**

In [24]:
# server_address = "https://localhost:5000"
server_address = "https://ltv5.microns-daf.com"
# server_address = "https://minniev6.microns-daf.com"

skclient = cc.skeletonservice.SkeletonClient(server_address, datastack_name, over_client=client, verify=False)

In [6]:
bulk_rids = [864691135463611454, 864691135687456480]
single_rid = bulk_rids[0]
sample_refusal_list_rid = 11223344556677889900
sample_invalid_node_rid = 864691135687000000
sample_supervoxel_rid = 88891049011371731
skvn = 4

# Delete the test rid files from the bucket so we can test regenerating them from scratch

In [43]:
from cloudfiles import CloudFiles

bucket = None
if "localhost" in server_address or "ltv" in server_address:
    bucket = f"gs://minnie65_skeletons/ltv/{skvn}"
elif "minnie" in server_address:
    bucket = f"gs://minnie65_skeletons/{skvn}"
print(f"Testing bucket: {bucket}")

cf = CloudFiles(bucket)
for rid in bulk_rids:
    for output_format in ["h5", "flatdict", "swccompressed"]:
        filename = f"skeleton__v{skvn}__rid-{rid}__ds-{datastack_name}__res-1x1x1__cs-True__cr-7500.{output_format}.gz"
        print(filename)
        print(cf.exists(filename))

skeleton__v4__rid-864691135397503777__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.h5.gz
True
skeleton__v4__rid-864691135397503777__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.flatdict.gz
True
skeleton__v4__rid-864691135397503777__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.swccompressed.gz
True
skeleton__v4__rid-864691135687456480__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.h5.gz
True
skeleton__v4__rid-864691135687456480__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.flatdict.gz
False
skeleton__v4__rid-864691135687456480__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.swccompressed.gz
False


In [44]:
from cloudfiles import CloudFiles

cf = CloudFiles(bucket)
for rid in bulk_rids:
    for output_format in ["h5", "flatdict", "swccompressed"]:
        filename = f"skeleton__v{skvn}__rid-{rid}__ds-{datastack_name}__res-1x1x1__cs-True__cr-7500.{output_format}.gz"
        print(filename)
        print(cf.exists(filename))
        cf.delete(filename)
        print(cf.exists(filename))

skeleton__v4__rid-864691135397503777__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.h5.gz
True
False
skeleton__v4__rid-864691135397503777__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.flatdict.gz
True
False
skeleton__v4__rid-864691135397503777__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.swccompressed.gz
True
False
skeleton__v4__rid-864691135687456480__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.h5.gz
True
False
skeleton__v4__rid-864691135687456480__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.flatdict.gz
False
False
skeleton__v4__rid-864691135687456480__ds-minnie65_phase3_v1__res-1x1x1__cs-True__cr-7500.swccompressed.gz
False
False


## Metadata tests

In [41]:
# %%time
precomputed_skeleton_info = skclient.get_precomputed_skeleton_info(skvn=skvn)
print(precomputed_skeleton_info)
assert precomputed_skeleton_info == {
    '@type': 'neuroglancer_skeletons',
    'transform': [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
    'vertex_attributes': [
        {'id': 'radius', 'data_type': 'float32', 'num_components': 1},
        {'id': 'compartment', 'data_type': 'uint8', 'num_components': 1}
    ]
}
print("TEST PASSED")

{'@type': 'neuroglancer_skeletons', 'transform': [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0], 'vertex_attributes': [{'id': 'radius', 'data_type': 'float32', 'num_components': 1}, {'id': 'compartment', 'data_type': 'uint8', 'num_components': 1}]}


In [45]:
import json

rids_exist = skclient.skeletons_exist(skeleton_version=skvn, root_ids=bulk_rids)
print(json.dumps(rids_exist, indent=4))
assert rids_exist == {
    bulk_rids[0]: False,
    bulk_rids[1]: False
}
print("TEST PASSED")

rids_exist = skclient.skeletons_exist(skeleton_version=skvn, root_ids=bulk_rids, log_warning=False)
print(json.dumps(rids_exist, indent=4))
assert rids_exist == {
    bulk_rids[0]: False,
    bulk_rids[1]: False
}
print("TEST PASSED")

rids_exist = skclient.skeletons_exist(skeleton_version=skvn, root_ids=bulk_rids, verbose_level=1)
print(json.dumps(rids_exist, indent=4))
assert rids_exist == {
    bulk_rids[0]: False,
    bulk_rids[1]: False
}
print("TEST PASSED")

INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]


{
    "864691135397503777": false,
    "864691135687456480": false
}


In [ ]:
import json

cache_contents = skclient.get_cache_contents(skeleton_version=skvn, root_id_prefixes=bulk_rids)
print(json.dumps(cache_contents, indent=4))
assert cache_contents == {
    "num_found": 0,
    "files": []
}
print("TEST PASSED")

cache_contents = skclient.get_cache_contents(skeleton_version=skvn, root_id_prefixes=bulk_rids, log_warning=False)
print(json.dumps(cache_contents, indent=4))
assert cache_contents == {
    "num_found": 0,
    "files": []
}
print("TEST PASSED")

cache_contents = skclient.get_cache_contents(skeleton_version=skvn, root_id_prefixes=bulk_rids, verbose_level=1)
print(json.dumps(cache_contents, indent=4))
assert cache_contents == {
    "num_found": 0,
    "files": []
}
print("TEST PASSED")

# Invalid skeleton request tests

In [3]:
# %%time
import requests

err = None
try:
    sk = skclient.get_skeleton(sample_refusal_list_rid, datastack_name, skeleton_version=skvn, output_format='dict', verbose_level=1)
except requests.HTTPError as e:
    print(e)
    assert e.response.text == '{\n    "Error": "Problematic root id: ' + str(sample_refusal_list_rid) + ' is in the refusal list"\n}\n'
print("TEST PASSED")

err = None
try:
    sk = skclient.get_skeleton(sample_invalid_node_rid, datastack_name, skeleton_version=skvn, output_format='dict', verbose_level=1)
except requests.HTTPError as e:
    print(e)
    assert e.response.text == '{\n    "Error": "Invalid root id: ' + str(sample_invalid_node_rid) + ' (perhaps it doesn\'t exist; the error is unclear)"\n}\n'
print("TEST PASSED")

err = None
try:
    sk = skclient.get_skeleton(sample_supervoxel_rid, datastack_name, skeleton_version=skvn, output_format='dict', verbose_level=1)
except requests.HTTPError as e:
    print(e)
    assert e.response.text == '{\n    "Error": "Invalid root id: ' + str(sample_supervoxel_rid) + ' (perhaps this is an id corresponding to a different level of the PCG, e.g., a supervoxel id)"\n}\n'
print("TEST PASSED")

/Users/keith.wiley/virtual_envs/work_env3/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


NameError: name 'skclient' is not defined

# Skeleton request tests

In [ ]:
%%time
sk = skclient.get_skeleton(single_rid, datastack_name, skeleton_version=skvn, output_format='dict')
# display(sk)
assert sk is not None and isinstance(sk, dict)
print("TEST PASSED")

In [32]:
%%time
sk = skclient.get_skeleton(single_rid, datastack_name, skeleton_version=skvn, output_format='dict', verbose_level=1)
# display(sk)
assert sk is not None and isinstance(sk, dict)
print("TEST PASSED")

INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]
INFO:root:SkeletonService version: 0.17.0
INFO:root:get_skeleton() response contains content of size 229266 bytes


{'meta': {'root_id': 864691135397503777,
  'soma_pt_x': 733312.0,
  'soma_pt_y': 433600.0,
  'soma_pt_z': 842280.0,
  'soma_radius': 7500,
  'collapse_soma': True,
  'collapse_function': 'sphere',
  'invalidation_d': 7500,
  'smooth_vertices': False,
  'compute_radius': False,
  'shape_function': 'single',
  'smooth_iterations': 12,
  'smooth_neighborhood': 2,
  'smooth_r': 0.1,
  'cc_vertex_thresh': 0,
  'remove_zero_length_edges': True,
  'collapse_params': {},
  'timestamp': 1741387327.3296018,
  'skeleton_type': 'pcg_skel',
  'meta': {'datastack': 'minnie65_phase3_v1', 'space': 'l2cache'},
  'sk_dict_structure_version': 4,
  'skeleton_version': 4},
 'edges': array([[3398, 3319],
        [3319, 3323],
        [3323, 3230],
        ...,
        [7113, 7110],
        [7110, 7106],
        [7106, 7107]]),
 'mesh_to_skel_map': array([   2,    0,    1, ..., 7470, 7472, 7473]),
 'root': 7474,
 'vertices': array([[ 482568.,  440648., 1002120.],
        [ 483456.,  440800., 1003600.],
     

CPU times: user 61.5 ms, sys: 31 ms, total: 92.6 ms
Wall time: 32.3 s


In [33]:
%%time
import pandas as pd

sk = skclient.get_skeleton(single_rid, datastack_name, skeleton_version=skvn, output_format='swc', verbose_level=1)
# display(sk)
assert sk is not None and isinstance(sk, pd.DataFrame)
print("TEST PASSED")

INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]
INFO:root:SkeletonService version: 0.17.0
INFO:root:get_skeleton() response contains content of size 108748 bytes


,id,type,x,y,z,radius,parent
0,0,1,733.312,433.600,842.28,5.851,-1
1,1,2,732.576,442.016,851.56,0.227,0
2,2,2,733.024,444.072,852.32,0.227,1
3,3,2,733.312,446.032,852.64,0.227,2
4,4,2,733.320,446.408,852.76,0.227,3
...,...,...,...,...,...,...,...
7470,7470,3,732.288,429.120,838.92,1.424,7469
7471,7471,3,734.416,429.304,838.08,1.424,7470
7472,7472,3,735.576,428.920,837.60,1.424,7471
7473,7473,3,736.368,428.432,837.08,1.424,7472


CPU times: user 37.6 ms, sys: 18 ms, total: 55.7 ms
Wall time: 1.47 s


### Inspect the cache after generating new skeletons

In [47]:
# %%time
rids_exist = skclient.skeletons_exist(skeleton_version=skvn, root_ids=bulk_rids)
print(json.dumps(rids_exist, indent=4))
assert rids_exist == {
    bulk_rids[0]: True,
    bulk_rids[1]: False
}
print("TEST PASSED")

INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]


{
    "864691135397503777": false,
    "864691135687456480": false
}


AssertionError: 

In [48]:
# %%time
import json
cache_contents = skclient.get_cache_contents(skeleton_version=skvn, root_id_prefixes=bulk_rids)
print(json.dumps(cache_contents, indent=4))
assert cache_contents == {
    "num_found": 1,
    "files": [
        f"skeleton__v4__rid-{bulk_rids[0]}__ds-{datastack_name}__res-1x1x1__cs-True__cr-7500.h5.gz"
    ]
}
print("TEST PASSED")

INFO:root:get_versions()
INFO:root:endpoint: {skeleton_server_address}/skeletoncache/api/versions
INFO:root:url: https://ltv5.microns-daf.com/skeletoncache/api/versions
INFO:root:response: <Response [200]>
INFO:root:versions: <class 'list'> [-1, 0, 1, 2, 3, 4]


{
    "num_found": 0,
    "files": []
}


AssertionError: 

# Bulk skeleton request tests

In [4]:
# %%time
result = skclient.get_bulk_skeletons(bulk_rids, skeleton_version=skvn, output_format='dict')
# We can't assert both root ids but only one was generated by the previous tests above.
# The other root id will be asyncronously triggered by this test but won't be available for 20-60 seconds afterwards.
assert(str(bulk_rids[0]) in result.keys())
print("TEST PASSED")

# %%time
result = skclient.get_bulk_skeletons(bulk_rids, skeleton_version=skvn, output_format='dict', verbose_level=1)
# We can't assert both root ids but only one was generated by the previous tests above.
# The other root id will be asyncronously triggered by this test but won't be available for 20-60 seconds afterwards.
assert(str(bulk_rids[0]) in result.keys())
print("TEST PASSED")

# %%time
result = skclient.generate_bulk_skeletons_async(bulk_rids, skeleton_version=skvn)
print(type(result), result)
assert result == 60.0
print("TEST PASSED")

# %%time
result = skclient.generate_bulk_skeletons_async(bulk_rids, skeleton_version=skvn, verbose_level=1)
print(type(result), result)
assert result == 60.0
print("TEST PASSED")

NameError: name 'skclient' is not defined

## Meshwork tests
### At the current time, I implemented meshwork generation and caching, but then removed it at the suggestion of other team members, saving the code for possible future use. Consequently, thge meshwork routines can't be tested until the code is added back at some later time. Therefore, the following tests are currently disabled.

In [37]:
RUN_MESHWORK_TESTS = False

In [38]:
%%time
from io import BytesIO

if RUN_MESHWORK_TESTS:
    import pcg_skel
    
    mw_bytes = skclient.get_meshwork(single_rid, datastack_name, verbose_level=1)
    print(len(mw_bytes))
    nrn = pcg_skel.meshwork.load_meshwork(BytesIO(mw_bytes))
    print(nrn)
    print(len(nrn.vertices), len(nrn.edges), len(nrn.anno['pre_syn']), len(nrn.anno['post_syn']))
    print(nrn.distance_to_root(nrn.anno.post_syn.mesh_index) / 1000)

    print("This test doesn't appear to be fully implemented yet, as it doesn't contain an assertion clause.")
    
    print("TEST PASSED")

CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 13.1 µs


In [39]:
%%time
if RUN_MESHWORK_TESTS:
    estimated_time = skclient.generate_bulk_meshworks_async(bulk_rids, datastack_name, verbose_level=1)
    print(type(result), result)
    assert result == 60.0
print("TEST PASSED")

CPU times: user 8 µs, sys: 3 µs, total: 11 µs
Wall time: 21 µs
